In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from sklearn.model_selection import train_test_split

NBA_learn_df=pd.read_csv("../Resources/NBA_learning2.csv")
NBA_learn_df

Using TensorFlow backend.


,Unnamed: 0,Year,Age,G,GS,MP,PTS,PER,TS%,FTr,FG,FGA,AST,PF,height,weight,Players ok,Played_Years
0,0,1986,22.0,82,1.0,1542.0,521,11.8,0.564,0.430,209,388,54,229,203,106,0,0
1,1,1987,23.0,79,72.0,2240.0,852,15.7,0.599,0.480,316,587,84,171,203,106,0,1
2,2,1988,24.0,82,64.0,2636.0,937,14.5,0.581,0.592,322,640,93,204,203,106,0,2
3,3,1989,25.0,82,82.0,2510.0,1088,17.8,0.594,0.474,401,758,103,172,203,106,0,3
4,4,1990,26.0,82,82.0,2709.0,1061,14.7,0.548,0.459,385,806,90,207,203,106,0,4
5,5,1991,27.0,82,21.0,2164.0,750,13.8,0.556,0.557,258,542,71,117,203,106,0,5
6,6,1992,28.0,82,53.0,2902.0,1116,16.7,0.556,0.569,382,803,117,141,203,106,0,6
7,7,1993,29.0,82,55.0,2819.0,1051,16.3,0.603,0.531,379,706,116,149,203,106,0,7
8,8,1994,30.0,82,55.0,2825.0,1204,17.0,0.555,0.391,465,926,137,142,203,106,0,8
9,9,1995,31.0,82,52.0,2687.0,916,14.2,0.596,0.556,311,617,127,146,203,106,0,9


In [2]:
NBA_learn_df.sample(frac=1).reset_index(drop=True)

,Unnamed: 0,Year,Age,G,GS,MP,PTS,PER,TS%,FTr,FG,FGA,AST,PF,height,weight,Players ok,Played_Years
0,11583,1966,25.0,79,NaN,3517.0,1697,18.8,0.499,0.265,690,1523,213,274,203,104,0,2
1,10909,2011,27.0,83,15.0,1722.0,735,14.2,0.503,0.336,260,637,242,102,190,90,0,5
2,15119,2010,25.0,9,0.0,61.0,36,18.3,0.473,0.200,14,35,5,5,190,86,0,2
3,15242,1986,30.0,44,1.0,682.0,198,8.5,0.508,0.085,92,188,61,128,203,108,0,9
4,3007,2017,31.0,41,2.0,416.0,142,9.8,0.510,0.320,46,122,52,18,185,78,0,6
5,5750,1989,24.0,77,36.0,1419.0,500,11.8,0.604,0.394,199,353,57,239,211,106,0,3
6,3367,1977,26.0,82,NaN,2023.0,494,12.3,0.520,0.274,215,424,92,301,211,98,0,0
7,18802,1986,28.0,64,63.0,2427.0,1632,19.1,0.547,0.309,633,1313,237,229,201,95,0,7
8,5755,1992,27.0,11,0.0,40.0,9,2.7,0.395,1.429,2,7,2,6,211,106,0,8
9,340,1968,24.0,81,NaN,2368.0,1060,12.0,0.480,0.266,437,989,111,262,203,86,0,0


In [14]:
X=NBA_learn_df.drop(columns=["Unnamed: 0","Players ok"])
y=NBA_learn_df['Players ok']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [15]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [16]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [64]:
import keras
print(keras.__version__)

2.3.0


In [17]:
mod1 = Sequential() 
mod1.add(Dense(50, activation='relu',input_shape=(16,)))
mod1.add(Dropout(0.3))
mod1.add(Dense(100, activation='relu'))
#mod1.add(Dropout(0.3))
mod1.add(Dense(50, activation='relu'))

mod1.add(Dense(1, activation="sigmoid"))

mod1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 50)                850       
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_7 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 51        
Total params: 11,051
Trainable params: 11,051
Non-trainable params: 0
_________________________________________________________________


In [88]:
from keras.optimizers import SGD
from keras import metrics
from keras.metrics import Recall
opt = SGD(lr=0.0001)
#model.compile(loss = "categorical_crossentropy", optimizer = opt)


mod1.compile(loss='binary_crossentropy', optimizer = opt, metrics=['binary_accuracy',Recall(thresholds=[0.3])])#thresholds=[0.5,0.5,0.5],


checkpointer = ModelCheckpoint(filepath='NBA.h5', monitor='loss',verbose=0, save_best_only='min', period=1)

h = mod1.fit(X_train_scaled, encoded_y_train, batch_size=1000, epochs=100, verbose=1, validation_data = (X_test_scaled,encoded_y_test),callbacks=[checkpointer])

Train on 14922 samples, validate on 4974 samples
Epoch 1/100
14922/14922 [==============================] - 0s 24us/step - loss: 0.1869 - binary_accuracy: 0.9283 - recall_12: 0.5282 - val_loss: 0.1629 - val_binary_accuracy: 0.9268 - val_recall_12: 0.5526
Epoch 2/100
14922/14922 [==============================] - 0s 7us/step - loss: 0.1887 - binary_accuracy: 0.9269 - recall_12: 0.5296 - val_loss: 0.1627 - val_binary_accuracy: 0.9268 - val_recall_12: 0.5766
Epoch 3/100
14922/14922 [==============================] - 0s 6us/step - loss: 0.1876 - binary_accuracy: 0.9286 - recall_12: 0.5223 - val_loss: 0.1625 - val_binary_accuracy: 0.9268 - val_recall_12: 0.5766
Epoch 4/100
14922/14922 [==============================] - 0s 6us/step - loss: 0.1860 - binary_accuracy: 0.9276 - recall_12: 0.5472 - val_loss: 0.1623 - val_binary_accuracy: 0.9268 - val_recall_12: 0.5766
Epoch 5/100
14922/14922 [==============================] - 0s 6us/step - loss: 0.1865 - binary_accuracy: 0.9270 - recall_12: 0.534

Epoch 80/100
14922/14922 [==============================] - 0s 6us/step - loss: 0.1759 - binary_accuracy: 0.9351 - recall_12: 0.6298 - val_loss: 0.1502 - val_binary_accuracy: 0.9403 - val_recall_12: 0.8206
Epoch 81/100
14922/14922 [==============================] - 0s 5us/step - loss: 0.1750 - binary_accuracy: 0.9347 - recall_12: 0.6394 - val_loss: 0.1501 - val_binary_accuracy: 0.9403 - val_recall_12: 0.8206
Epoch 82/100
14922/14922 [==============================] - 0s 6us/step - loss: 0.1754 - binary_accuracy: 0.9356 - recall_12: 0.6459 - val_loss: 0.1500 - val_binary_accuracy: 0.9403 - val_recall_12: 0.8206
Epoch 83/100
14922/14922 [==============================] - 0s 6us/step - loss: 0.1759 - binary_accuracy: 0.9344 - recall_12: 0.6379 - val_loss: 0.1499 - val_binary_accuracy: 0.9403 - val_recall_12: 0.8206
Epoch 84/100
14922/14922 [==============================] - 0s 5us/step - loss: 0.1736 - binary_accuracy: 0.9353 - recall_12: 0.6467 - val_loss: 0.1497 - val_binary_accuracy: 0

In [89]:
model_loss, model_accuracy,recall = mod1.evaluate(
    X_test_scaled, encoded_y_test, verbose=0)
print("Normal Neural Network - Loss:" +str(model_loss)+", Accuracy: "+str(model_accuracy)+ " Recall: "+str(recall))
#mod1.evaluate(    X_test_scaled, encoded_y_test, verbose=0)

Normal Neural Network - Loss:0.1478075708161888, Accuracy: 0.9402894973754883 Recall: 0.8492823243141174


In [90]:
encoded_predictions = mod1.predict_classes(X_test_scaled[:1000])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

C:\Users\Joshua Castillo\AppData\Local\conda\conda\envs\actumlogos-gpu\lib\site-packages\sklearn\preprocessing\label.py:273: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [91]:
print("Predicted classes:"+ str(prediction_labels))
print("Actual Labels: "+str(list(y_test[:1000])))

Predicted classes:[1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 

In [92]:
NBA_learn_df=NBA_learn_df.dropna()
X=NBA_learn_df.drop(columns=["Unnamed: 0","PTS"])
y=NBA_learn_df['PTS']


In [93]:


X_scaler = MinMaxScaler().fit(X)
X_scaled = X_scaler.transform(X)




y=pd.DataFrame(y)
y_scaler = MinMaxScaler().fit(y)#.reshape(-1, 1)
y_scaled = y_scaler.transform(y)




X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, random_state=1)

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [94]:
mod2 = Sequential() 
mod2.add(Dense(50, activation='relu',input_shape=(16,)))
#mod2.add(Dropout(0.3))
mod2.add(Dense(100, activation='relu'))
#mod2.add(Dropout(0.3))
mod2.add(Dense(50, activation='relu'))

mod2.add(Dense(1, activation="relu"))

mod2.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 50)                850       
_________________________________________________________________
dense_14 (Dense)             (None, 100)               5100      
_________________________________________________________________
dense_15 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 51        
Total params: 11,051
Trainable params: 11,051
Non-trainable params: 0
_________________________________________________________________


In [114]:
from keras import metrics
#from keras.metrics import MeanRelativeError

opt = SGD(lr=0.1)
mod2.compile(loss="mse", optimizer = opt,metrics=["mean_absolute_percentage_error"])#, metrics=['accuracy']

checkpointer = ModelCheckpoint(filepath='NBA2.h5', monitor='loss',verbose=0, save_best_only='min', period=1)

h = mod2.fit(X_train, y_train, batch_size=1000, epochs=1000, verbose=1, validation_data = (X_test,y_test),callbacks=[checkpointer])

Train on 14922 samples, validate on 4974 samples
Epoch 1/1000
14922/14922 [==============================] - 0s 12us/step - loss: 1.8580e-04 - mean_absolute_percentage_error: 25.1782 - val_loss: 1.7691e-04 - val_mean_absolute_percentage_error: 26.0786
Epoch 2/1000
14922/14922 [==============================] - 0s 5us/step - loss: 1.8529e-04 - mean_absolute_percentage_error: 25.1221 - val_loss: 1.7779e-04 - val_mean_absolute_percentage_error: 23.5282
Epoch 3/1000
14922/14922 [==============================] - 0s 5us/step - loss: 1.6502e-04 - mean_absolute_percentage_error: 24.1398 - val_loss: 1.8279e-04 - val_mean_absolute_percentage_error: 26.3782
Epoch 4/1000
14922/14922 [==============================] - 0s 5us/step - loss: 1.7365e-04 - mean_absolute_percentage_error: 24.7106 - val_loss: 1.5915e-04 - val_mean_absolute_percentage_error: 22.9372
Epoch 5/1000
14922/14922 [==============================] - 0s 5us/step - loss: 1.5268e-04 - mean_absolute_percentage_error: 23.6634 - val_los

14922/14922 [==============================] - 0s 4us/step - loss: 1.4709e-04 - mean_absolute_percentage_error: 23.3338 - val_loss: 1.4862e-04 - val_mean_absolute_percentage_error: 22.4999
Epoch 82/1000
14922/14922 [==============================] - 0s 5us/step - loss: 1.5690e-04 - mean_absolute_percentage_error: 23.8899 - val_loss: 1.3956e-04 - val_mean_absolute_percentage_error: 22.6483
Epoch 83/1000
14922/14922 [==============================] - 0s 4us/step - loss: 1.5156e-04 - mean_absolute_percentage_error: 23.4477 - val_loss: 1.7154e-04 - val_mean_absolute_percentage_error: 25.6766
Epoch 84/1000
14922/14922 [==============================] - 0s 4us/step - loss: 1.5390e-04 - mean_absolute_percentage_error: 23.8600 - val_loss: 1.4164e-04 - val_mean_absolute_percentage_error: 22.4056
Epoch 85/1000
14922/14922 [==============================] - 0s 4us/step - loss: 1.5663e-04 - mean_absolute_percentage_error: 23.6533 - val_loss: 1.5964e-04 - val_mean_absolute_percentage_error: 24.6115

14922/14922 [==============================] - 0s 4us/step - loss: 1.4464e-04 - mean_absolute_percentage_error: 23.0273 - val_loss: 1.3604e-04 - val_mean_absolute_percentage_error: 22.2015
Epoch 162/1000
14922/14922 [==============================] - 0s 4us/step - loss: 1.5511e-04 - mean_absolute_percentage_error: 23.5988 - val_loss: 1.6321e-04 - val_mean_absolute_percentage_error: 22.7015
Epoch 163/1000
14922/14922 [==============================] - 0s 4us/step - loss: 1.4640e-04 - mean_absolute_percentage_error: 23.1145 - val_loss: 1.3556e-04 - val_mean_absolute_percentage_error: 22.4892
Epoch 164/1000
14922/14922 [==============================] - 0s 4us/step - loss: 1.4603e-04 - mean_absolute_percentage_error: 23.1979 - val_loss: 1.6243e-04 - val_mean_absolute_percentage_error: 25.0598
Epoch 165/1000
14922/14922 [==============================] - 0s 4us/step - loss: 1.4446e-04 - mean_absolute_percentage_error: 23.2789 - val_loss: 1.3555e-04 - val_mean_absolute_percentage_error: 22.

14922/14922 [==============================] - 0s 5us/step - loss: 1.3320e-04 - mean_absolute_percentage_error: 22.4102 - val_loss: 1.3450e-04 - val_mean_absolute_percentage_error: 22.9414
Epoch 242/1000
14922/14922 [==============================] - 0s 4us/step - loss: 1.4474e-04 - mean_absolute_percentage_error: 22.9880 - val_loss: 1.3667e-04 - val_mean_absolute_percentage_error: 21.8112
Epoch 243/1000
14922/14922 [==============================] - 0s 5us/step - loss: 1.3382e-04 - mean_absolute_percentage_error: 22.4432 - val_loss: 1.3400e-04 - val_mean_absolute_percentage_error: 22.8999
Epoch 244/1000
14922/14922 [==============================] - 0s 5us/step - loss: 1.5063e-04 - mean_absolute_percentage_error: 23.5714 - val_loss: 1.4867e-04 - val_mean_absolute_percentage_error: 22.0926
Epoch 245/1000
14922/14922 [==============================] - 0s 5us/step - loss: 1.3864e-04 - mean_absolute_percentage_error: 22.5759 - val_loss: 1.3552e-04 - val_mean_absolute_percentage_error: 21.

14922/14922 [==============================] - 0s 4us/step - loss: 1.3614e-04 - mean_absolute_percentage_error: 22.3784 - val_loss: 1.4190e-04 - val_mean_absolute_percentage_error: 23.6305
Epoch 322/1000
14922/14922 [==============================] - 0s 5us/step - loss: 1.3279e-04 - mean_absolute_percentage_error: 22.1663 - val_loss: 1.3027e-04 - val_mean_absolute_percentage_error: 21.4971
Epoch 323/1000
14922/14922 [==============================] - 0s 4us/step - loss: 1.4606e-04 - mean_absolute_percentage_error: 22.8153 - val_loss: 1.7249e-04 - val_mean_absolute_percentage_error: 26.0346
Epoch 324/1000
14922/14922 [==============================] - 0s 5us/step - loss: 1.4424e-04 - mean_absolute_percentage_error: 23.1098 - val_loss: 1.3353e-04 - val_mean_absolute_percentage_error: 21.5190
Epoch 325/1000
14922/14922 [==============================] - 0s 5us/step - loss: 1.4801e-04 - mean_absolute_percentage_error: 23.1147 - val_loss: 1.5143e-04 - val_mean_absolute_percentage_error: 24.

...

14922/14922 [==============================] - 0s 4us/step - loss: 1.4059e-04 - mean_absolute_percentage_error: 22.2353 - val_loss: 1.4217e-04 - val_mean_absolute_percentage_error: 21.1109
Epoch 642/1000
14922/14922 [==============================] - 0s 4us/step - loss: 1.4011e-04 - mean_absolute_percentage_error: 21.9511 - val_loss: 1.3030e-04 - val_mean_absolute_percentage_error: 22.7008
Epoch 643/1000
14922/14922 [==============================] - 0s 4us/step - loss: 1.2449e-04 - mean_absolute_percentage_error: 21.2429 - val_loss: 1.1618e-04 - val_mean_absolute_percentage_error: 20.6027
Epoch 644/1000
14922/14922 [==============================] - 0s 4us/step - loss: 1.3401e-04 - mean_absolute_percentage_error: 21.7744 - val_loss: 1.1807e-04 - val_mean_absolute_percentage_error: 21.5901
Epoch 645/1000
14922/14922 [==============================] - 0s 4us/step - loss: 1.1677e-04 - mean_absolute_percentage_error: 20.7635 - val_loss: 1.1378e-04 - val_mean_absolute_percentage_error: 20.

14922/14922 [==============================] - 0s 5us/step - loss: 2.1362e-04 - mean_absolute_percentage_error: 25.3519 - val_loss: 1.8434e-04 - val_mean_absolute_percentage_error: 22.5915
Epoch 722/1000
14922/14922 [==============================] - 0s 4us/step - loss: 1.8288e-04 - mean_absolute_percentage_error: 23.5947 - val_loss: 1.5877e-04 - val_mean_absolute_percentage_error: 24.4542
Epoch 723/1000
14922/14922 [==============================] - 0s 4us/step - loss: 1.2679e-04 - mean_absolute_percentage_error: 21.2411 - val_loss: 1.2639e-04 - val_mean_absolute_percentage_error: 20.3869
Epoch 724/1000
14922/14922 [==============================] - 0s 4us/step - loss: 1.2564e-04 - mean_absolute_percentage_error: 21.2265 - val_loss: 1.2013e-04 - val_mean_absolute_percentage_error: 21.5473
Epoch 725/1000
14922/14922 [==============================] - 0s 4us/step - loss: 1.2099e-04 - mean_absolute_percentage_error: 20.6322 - val_loss: 1.4433e-04 - val_mean_absolute_percentage_error: 23.

14922/14922 [==============================] - 0s 5us/step - loss: 1.6566e-04 - mean_absolute_percentage_error: 23.2744 - val_loss: 1.2542e-04 - val_mean_absolute_percentage_error: 20.3074
Epoch 802/1000
14922/14922 [==============================] - 0s 4us/step - loss: 1.2117e-04 - mean_absolute_percentage_error: 20.6008 - val_loss: 1.2476e-04 - val_mean_absolute_percentage_error: 22.0457
Epoch 803/1000
14922/14922 [==============================] - 0s 4us/step - loss: 1.1453e-04 - mean_absolute_percentage_error: 20.3134 - val_loss: 1.2808e-04 - val_mean_absolute_percentage_error: 20.3900
Epoch 804/1000
14922/14922 [==============================] - 0s 5us/step - loss: 1.2353e-04 - mean_absolute_percentage_error: 20.6142 - val_loss: 1.5517e-04 - val_mean_absolute_percentage_error: 24.4187
Epoch 805/1000
14922/14922 [==============================] - 0s 5us/step - loss: 1.9572e-04 - mean_absolute_percentage_error: 24.6213 - val_loss: 1.5241e-04 - val_mean_absolute_percentage_error: 21.

14922/14922 [==============================] - 0s 4us/step - loss: 1.3176e-04 - mean_absolute_percentage_error: 21.0444 - val_loss: 1.5358e-04 - val_mean_absolute_percentage_error: 24.0369
Epoch 882/1000
14922/14922 [==============================] - 0s 4us/step - loss: 1.9231e-04 - mean_absolute_percentage_error: 24.0705 - val_loss: 2.1640e-04 - val_mean_absolute_percentage_error: 23.6222
Epoch 883/1000
14922/14922 [==============================] - 0s 4us/step - loss: 1.8064e-04 - mean_absolute_percentage_error: 23.9706 - val_loss: 1.4203e-04 - val_mean_absolute_percentage_error: 22.9371
Epoch 884/1000
14922/14922 [==============================] - 0s 4us/step - loss: 1.1455e-04 - mean_absolute_percentage_error: 20.4062 - val_loss: 1.0515e-04 - val_mean_absolute_percentage_error: 19.9386
Epoch 885/1000
14922/14922 [==============================] - 0s 4us/step - loss: 1.0542e-04 - mean_absolute_percentage_error: 19.5664 - val_loss: 1.0804e-04 - val_mean_absolute_percentage_error: 20.

14922/14922 [==============================] - 0s 4us/step - loss: 1.9254e-04 - mean_absolute_percentage_error: 23.8639 - val_loss: 2.2978e-04 - val_mean_absolute_percentage_error: 28.8745
Epoch 962/1000
14922/14922 [==============================] - 0s 4us/step - loss: 1.4748e-04 - mean_absolute_percentage_error: 21.9052 - val_loss: 1.0694e-04 - val_mean_absolute_percentage_error: 19.5581
Epoch 963/1000
14922/14922 [==============================] - 0s 4us/step - loss: 1.0662e-04 - mean_absolute_percentage_error: 19.5521 - val_loss: 1.0488e-04 - val_mean_absolute_percentage_error: 20.1438
Epoch 964/1000
14922/14922 [==============================] - 0s 4us/step - loss: 1.0630e-04 - mean_absolute_percentage_error: 19.5838 - val_loss: 1.1000e-04 - val_mean_absolute_percentage_error: 19.6711
Epoch 965/1000
14922/14922 [==============================] - 0s 4us/step - loss: 1.2188e-04 - mean_absolute_percentage_error: 20.4236 - val_loss: 1.1519e-04 - val_mean_absolute_percentage_error: 21.

In [115]:
model_loss, model_accuracy = mod2.evaluate(
    X_test, y_test, verbose=0)
print("Normal Neural Network - val_loss:" +str(model_loss)+", val_mean_absolute_percentage_error: "+str(model_accuracy))

Normal Neural Network - val_loss:0.00012125356985399032, val_mean_absolute_percentage_error: 20.101566314697266


In [116]:
 mod2.evaluate(X_test, y_test, verbose=0)

[0.00012125356985399032, 20.101566314697266]